In [1]:
# Crear una instancia del ambiente
from template import AgenteQLearning, AmbienteDiezMil
import matplotlib.pyplot as plt
import numpy as np
seed = 42
np.random.seed(seed)

ambiente = AmbienteDiezMil()

# Crear un agente de Q-learning
agente = AgenteQLearning(ambiente)
episodios = 10000

# Entrenar al agente con un número de episodios
vals = list(agente.entrenar(episodios, verbose=True))
# agente.guardar_politica(f"politica_{episodios}.csv")

100%|██████████| 10000/10000 [00:05<00:00, 1733.97it/s]


In [2]:
from jugador import JugadorAleatorio, JugadorSiempreSePlanta
from template import Validador
np.random.seed(seed)

val = Validador(ambiente)
val_count = 1000
avg = val.validar_politica(agente.q_table2pol(), val_count)
print(f"Validación DeepPurple: {avg}")

val_rand = Validador(ambiente)
jugador = JugadorAleatorio("random")
avg_rand = val_rand.validar_jugador(jugador, val_count)
print(f"Validación Random: {avg_rand}")

val_planton = Validador(ambiente)
jugador = JugadorSiempreSePlanta("plantón")
avg_planton = val_planton.validar_jugador(jugador, val_count)
print(f"Validación Plantón: {avg_planton}")

Validación DeepPurple: 25.393
Validación Random: 27.034
Validación Plantón: 25.143


In [3]:
# Hyperparameter tuning

# alpha: float = 0.1,
# gamma: float = 0.9,
# epsilon: float = 0.1,

paramter_space = {
    "alpha": np.arange(0.1, 1.1, 0.1),
    "gamma": np.arange(0.0, 1.2, 0.2),
    "epsilon": np.arange(0.05, 0.25, 0.05),
    "reward_func": list(range(0, 9))
}
val_count = 50

best_avg = 35
best_params = None

# Random search
from tqdm import tqdm
iter = tqdm(range(100))
for i in iter:
    alpha = np.random.choice(paramter_space["alpha"])
    gamma = np.random.choice(paramter_space["gamma"])
    epsilon = np.random.choice(paramter_space["epsilon"])
    reward_func = np.random.choice(paramter_space["reward_func"])

    ambiente = AmbienteDiezMil(reward_func_type=reward_func)
    agente = AgenteQLearning(ambiente, alpha=alpha, gamma=gamma, epsilon=epsilon)
    vals = list(agente.entrenar(episodios))
    avg = val.validar_politica(agente.q_table2pol(), val_count)

    if best_avg > avg:
        best_avg = avg
        best_params = {
            "alpha": alpha,
            "gamma": gamma,
            "epsilon": epsilon,
            "reward_func": reward_func
        }
        iter.set_description(f"Best params: {best_params}, avg: {best_avg}")

print(f"Best params: {best_params}, avg: {best_avg}")

Best params: {'alpha': np.float64(0.4), 'gamma': np.float64(0.2), 'epsilon': np.float64(0.05), 'reward_func': np.int64(5)}, avg: 23.18: 100%|██████████| 100/100 [01:02<00:00,  1.61it/s]                            

Best params: {'alpha': np.float64(0.4), 'gamma': np.float64(0.2), 'epsilon': np.float64(0.05), 'reward_func': np.int64(5)}, avg: 23.18


In [15]:
ambiente = AmbienteDiezMil(reward_func_type=best_params["reward_func"])

# Crear un agente de Q-learning
agente = AgenteQLearning(ambiente)
episodios = 100000

# Entrenar al agente con un número de episodios
list(agente.entrenar(episodios, verbose=True))

val = Validador(ambiente)
val_count = 1000
avg = val.validar_politica(agente.q_table2pol(), val_count)
print(f"Validación DeepPurple: {avg}")

 57%|█████▋    | 57236/100000 [00:42<00:31, 1338.00it/s]


KeyboardInterrupt: 

In [ ]:
# !pip install hyperopt

In [10]:
# HyperOpt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(params):
    alpha = params["alpha"]
    gamma = params["gamma"]
    epsilon = params["epsilon"]

    ambiente = AmbienteDiezMil(reward_func_type=7)

    agente = AgenteQLearning(ambiente, alpha=alpha, gamma=gamma, epsilon=epsilon)
    agente.entrenar(episodios)
    avg = val.validar_politica(agente.q_table2pol(), val_count)

    return {
        "loss": avg,
        "status": STATUS_OK,
    }

space = {
    "alpha": hp.uniform("alpha", 0.1, 1.0),
    "gamma": hp.uniform("gamma", 0.0, 1.0),
    "epsilon": hp.uniform("epsilon", 0.05, 0.25),
}

trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)
print(best)

# Entrenar al agente con un número de episodios
agente = AgenteQLearning(ambiente, **best)
vals = list(agente.entrenar(episodios))

100%|██████████| 100/100 [00:02<00:00, 47.32trial/s, best loss: 23.02]
{'alpha': np.float64(0.7948009830124299), 'epsilon': np.float64(0.07536342092227688), 'gamma': np.float64(0.9998031968152667)}
